In [ ]:
from a1chemy.data_source import SinaFinanceClient, EastMoneyClient
from a1chemy.common import OptionConfig, OptionMap, Underlying
from a1chemy.option.pricing import black_scholes, black_76, gbs
import datetime

east_money_client = EastMoneyClient()


symbol = 'SH510300'
underlying = Underlying(price=5.338)
config = OptionConfig(interest_rate=0.029120, pricing_model=gbs)
option_map = OptionMap()
expiry_list = [
    '2021-06-23',
    '2021-07-28',
    '2021-09-22',
    '2021-12-22'
]

expiry_date_list = [datetime.date.fromisoformat(expiry) for expiry in expiry_list]

for time_to_expiry in expiry_date_list:
    east_money_client.get_option_data(underlying, config, option_map, 'SH510300', time_to_expiry)

In [34]:
# for expire_date, chain in option_map.index.items():
#     print(expire_date)
#     for strike, straddle in chain.index.items():
#         print('{} {} {} {}'.format(expire_date, straddle.strike, straddle.call.price, straddle.put.price))


def calculate_positions(positions):
    delta = 0
    delta_without_underlying = 0
    gamma = 0
    theta = 0
    vega = 0

    for p in positions:
        if p[0] == 'underlying':
            delta += p[1]
        else:
            o = option_map.get_option(p[2], p[3], p[1])
            delta += p[4] * o.delta
            delta_without_underlying += p[4] * o.delta
            gamma += p[4] * o.gamma
            theta += p[4] * o.theta
            vega += p[4] * o.vega
    return delta, delta_without_underlying, gamma, theta, vega

positions = [
    ('underlying', 50000),
    ('option', 'c', expiry_date_list[0], 5.5, -20000),
    ('option', 'p', expiry_date_list[0], 5.25, -10000),
    ('option', 'c', expiry_date_list[1], 5.5, 10000),
    ('option', 'c', expiry_date_list[1], 5.75, -10000),
]

delta, delta_without_underlying, gamma, theta, vega = calculate_positions(positions)
print('delta:{}({}), gamma:{} theta:{} vega:{}'.format(delta, delta_without_underlying, gamma, theta, vega))

delta:48673.701891931305(-1326.2981080686955), gamma:-34119.28226570149 theta:24339.984105034287 vega:-13396.083966412974
